# Cálculo del punto central con la mayor concentración geográfica

In [5]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

Cargamos los datos:

In [6]:
df = pd.read_feather('../data/03_model_input/california_housing_clean.ftr')
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


Creamos una matriz de coordenadas (lat, lon):

In [7]:
coords = df[['Latitude', 'Longitude']].to_numpy()

Calculamos la matriz de distancias en grados

In [8]:
distancias = cdist(coords, coords, metric='euclidean')
distancias

array([[0.        , 0.02236068, 0.03162278, ..., 1.85002703, 1.79738699,
        1.78891028],
       [0.02236068, 0.        , 0.02236068, ..., 1.86142419, 1.80966848,
        1.80013888],
       [0.03162278, 0.02236068, 0.        , ..., 1.88063819, 1.82833257,
        1.81945047],
       ...,
       [1.85002703, 1.86142419, 1.88063819, ..., 0.        , 0.1       ,
        0.06324555],
       [1.79738699, 1.80966848, 1.82833257, ..., 0.1       , 0.        ,
        0.1       ],
       [1.78891028, 1.80013888, 1.81945047, ..., 0.06324555, 0.1       ,
        0.        ]], shape=(19128, 19128))

Sumamos las viviendas de todos los distritos a menos de 500 metros

In [ ]:
matrix_cumulos_500m = distancias < 500 / 100000  # Aproximación de 500 metros a 0.005 grados
matrix_cumulos_500m

In [ ]:
matrix_capitales = matrix_cumulos_500m * df['target'].values[np.newaxis, :]
matrix_capitales

array([[4.526, 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 3.585, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 3.521, ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.923, 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.847, 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.894]])

In [ ]:
capital_concentrado = matrix_capitales.sum(axis=1)

Encontramos el índice del punto con mayor concentración

In [ ]:
indice_max = np.argmax(capital_concentrado)
punto_central = df.iloc[indice_max]
punto_central

,14569
MedInc,2.856900
HouseAge,52.000000
AveRooms,4.703316
AveBedrms,1.146597
Population,1243.000000
AveOccup,2.169284
Latitude,37.780000
Longitude,-122.440000
target,3.722000


Coordenadas del punto más denso en contador de viviendas

In [ ]:
print(f'Latitud: {punto_central['Latitude']}, Longitud: {punto_central['Longitude']}')
print(f'Capital total radio 500 m: ${capital_concentrado[indice_max]:,.0f}')
print(f'Número de viviendas del cúmulo: {matrix_cumulos_500m[indice_max].sum()}')

Coordenadas del punto con mayor capital concentrado:
Latitud: 37.78000, Longitud: -122.44000
Capital total a menos de 500 m: $39
Número de viviendas cercanas: 11
Valor medio por distrito cercano: $4
